In [ ]:
!pip install streamlit
!npm install localtunnel

In [ ]:
%%writefile app.py
import re
import streamlit as st
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import wandb
import warnings

wandb.init(mode="disabled")
warnings.filterwarnings('ignore')

model_path = "/kaggle/input/custom-model/custom_model"

def load_model(model_path):
    model = GPT2LMHeadModel.from_pretrained(model_path)
    return model

def load_tokenizer(tokenizer_path):
    tokenizer = GPT2Tokenizer.from_pretrained(tokenizer_path)
    return tokenizer

def generate_text(model_path, sequence, max_length=200):
    model = load_model(model_path)
    tokenizer = load_tokenizer(model_path)
    ids = tokenizer.encode(f'{sequence}', return_tensors='pt')
    final_outputs = model.generate(
        ids,
        do_sample=True,
        max_length=max_length,
        pad_token_id=model.config.eos_token_id,
        top_k=50,
        top_p=0.7,
        temperature=0.5
    )
    generated_text = tokenizer.decode(final_outputs[0], skip_special_tokens=True)
    
    # Remove the input sequence from the generated text
    if generated_text.lower().startswith(sequence.lower()):
        generated_text = generated_text[len(sequence):].strip()
        
    match = re.search(r'\d', generated_text)
    if match:
        generated_text = generated_text[:match.start()].strip()
    
    return generated_text    
    

# Streamlit app
st.set_page_config(page_title="ChatBot", page_icon="🤖", layout="centered", initial_sidebar_state="expanded")

st.markdown(
    """
    <style>
    .main {
        background-color: #f5f5f5;
        padding: 20px;
        border-radius: 10px;
    }
    .stButton button {
        background-color: #4CAF50;
        color: white;
        border: none;
        padding: 10px 20px;
        text-align: center;
        text-decoration: none;
        display: inline-block;
        font-size: 16px;
        margin: 4px 2px;
        border-radius: 16px;
        transition-duration: 0.4s;
    }
    .stButton button:hover {
        background-color: white;
        color: black;
        border: 2px solid #4CAF50;
    }
    .message {
        border: 1px solid #d3d3d3;
        padding: 10px;
        border-radius: 10px;
        margin: 10px 0;
    }
    .user_message {
        background-color: #e1f5fe;
    }
    .bot_message {
        background-color: #fff9c4;
    }
    </style>
    """,
    unsafe_allow_html=True
)

st.title("🤖 Medical ChatBot")
st.write("Welcome to the interactive ChatBot about healthcare questions. Type a message below and get a response.")

user_input = st.text_input("You:", placeholder="Type your message here...")

if st.button("Send"):
    if user_input:
        with st.spinner("Generating response..."):
            for i in range (0,2):
                response = generate_text(model_path, user_input)
            st.markdown(
                f"""
                <div class="message user_message">
                    <strong>You:</strong> {user_input}
                </div>
                <div class="message bot_message">
                    <strong>Bot:</strong> {response}
                </div>
                """,
                unsafe_allow_html=True
            )
    else:
        st.write("Please enter a message to get a response.")
else:
    st.write("Awaiting your message...")

st.markdown(
    """
    <div style="text-align: center; margin-top: 20px;">
        <p>Developed using Streamlit</p>
    </div>
    """,
    unsafe_allow_html=True
)

In [ ]:
#Your public ip is the password to the localtunnel
!curl ipv4.icanhazip.com

In [ ]:
!streamlit run app.py &>./logs.txt & npx localtunnel --port 8501